In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                  dcc.Dropdown(id='site-dropdown',
                                  options=[
                                      {'label': 'All Sites', 'value': 'ALL'},
                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                      ],
                                      value='ALL',
                                      placeholder="place holder here",
                                      searchable=True
                                      ),
                                html.Br(),
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       2500: '2500',
                                                      5000: '5000',
                                                      10000: '10000'},
                                                value=[0, 10000]),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                               ])



# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', #to count from a different column
        names='Launch Site', 
        title='Total number of launches per site')
        return fig 
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, 
                             names='class', 
                             title='Total number of launches for selected site')
        return fig 

# TASK 4: Add a scatter chart to show the correlation between payload and launch success

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
[Input(component_id='site-dropdown', component_property='value'),
 Input(component_id="payload-slider", component_property="value")
])
def get_scatter(entered_site, slider_range):
    low,high = slider_range
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    if entered_site == 'ALL':
        fig= px.scatter(
            filtered_df, x="Payload Mass (kg)", y="class",
            color="Booster Version Category",
            hover_data=['Booster Version'],
            title="Correlation between Payload and Success for all Sites")
        return fig 
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig=px.scatter(
                    filtered_df,x='Payload Mass (kg)', y='class', 
                    title = 'Success by Payload for selected site', 
                    color='Booster Version Category')
        return fig 


# Run the app
if __name__ == '__main__':
    app.run_server()

Dash app running on http://127.0.0.1:8050/


In [3]:
total = spacex_df['class'].sum()
spacex_df['class'].mean()
spacex_df.columns

Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')